# Teaching an AI to Play Pong with Reinforcement Learning
***

### Intro

* See the accompanying slide deck: [link to deck here]

* This code and discussion borrows from Andrej Karpathy's excellent blog post, ["Pong from Pixels"](http://karpathy.github.io/2016/05/31/rl/).

* If you want to take a step back and cover some basic RL concepts, [check out this blog post](https://blog.floydhub.com/an-introduction-to-q-learning-reinforcement-learning/).

* There are several approaches to reinforcement learning (e.g. [some are described here](https://www.kdnuggets.com/2018/03/5-things-reinforcement-learning.html)). Below we're going to focus on the **policy gradient** approach.    

* We use the [OpenAI gym library](http://gym.openai.com/docs/) for our Pong simulator. 

### Setup

In [ ]:
import numpy as np
import gym
import pickle

In [ ]:
!pip install gym[atari]

***
## Training (Policy Gradient Approach)
### Model Hyperparameters and Training Options

In [ ]:
# *** MODEL HYPERPARAMS ***
H = 200 # number of hidden layer neurons
batch_size = 10 # number of episodes before a parameter update
learning_rate = 1e-4
gamma = 0.99 # discount factor for reward
decay_rate = 0.99 # decay factor for RMSProp leaky sum of grad^2

# *** TRAINING OPTIONS *** 
save_rate = 100 # save model after every 'x' episodes
resume = False # resume from previous checkpoint?
render = False

# *** INITIALIZATION ***
D = 80 * 80 # input dimensionality: 80x80 grid
if resume:
    model = pickle.load(open('save.p', 'rb'))
else: 
    model = {}
    # first layer is a matrix that maps 80*80 inputs (pixels) to H outputs (hidden units)
    model['W1'] = np.random.randn(H,D) / np.sqrt(D) # "Xavier" initialization
    # second layer here is just a vector of H weights (we'll apply a sigmoid to convert this into a probability)
    model['W2'] = np.random.randn(H) / np.sqrt(H)

As per [Karpathy](http://karpathy.github.io/2016/05/31/rl/), we'll use a two-layer fully-connected net. Intuitively the first layer weights `W1` learn to recognize various game scenarios (e.g. ball at the top, our paddle in the middle) and the second layer weights `W2` can then decide in each case whether we should go UP or DOWN. Each path/line in the image below represents one tunable weight in our network. The nodes in this case represent us applying a non-linearity $f(x)$. 

<img src="./img/pong-net.png" alt="Drawing" style="width: 400px;"/>

Mathematical Representation:
\begin{align*}
h &= f(W_{1}X)\\
logit &= W_{2}h \\
P_{\mathrm{up}} &= \sigma (logit)
\end{align*}

* $X$ is a flattened vector of input pixels
* $f(x)$ is our non-linearity, which will be the [Rectified Linear Unit](https://machinelearningmastery.com/rectified-linear-activation-function-for-deep-learning-neural-networks/) (RELU), $f(x) = max(0, x)$
* $\sigma(x)$ is the [sigmoid function](https://www.sciencedirect.com/topics/computer-science/sigmoid-function), which maps its input to a probability between 0 and 1:

\begin{equation*}
\sigma(x) = \frac{e^{x}}{e^{x} + 1}
\end{equation*}

Note that the outputs of the final layer (prior to the sigmoid) are often called logits, which is sometimes abbreviated as `logp`, not be confused with the mathematical expression log(P). 

### Forward Propagation
Here's the code that evolves our network forward:

In [ ]:
# *** convert our game pixels into a format that our neural network will understand
def prepro(I):
    """ preproccess 210x160x3 uint8 game frame into 6400 (80x80) 1D float vector """
    I = I[35:195] # crop
    I = I[::2,::2,0] # downsample by factor of 2
    I[I == 144] = 0 # erase background (background type 1)
    I[I == 109] = 0 # erase background (background type 2)
    I[I != 0] = 1 # everything else (paddles, ball) just set to 1
    return I.astype(np.float).ravel()

# *** helper function to convert our logits into probabilities
def sigmoid(x): 
    return 1.0 / (1.0 + np.exp(-x)) # sigmoid "squashing" function to interval [0,1]

# *** computing the probability of up, and the hidden states h, for this game frame
def policy_forward(x):
    h = np.dot(model['W1'], x)
    h[h<0] = 0 # ReLU nonlinearity
    logp = np.dot(model['W2'], h)
    p = sigmoid(logp)
    return p, h # return probability of taking action 2, and hidden state


### Back-Propagation

Here's a recap of how our approach will work. For each game frame, our model computes the probability that it should move the paddle DOWN or UP. It rolls a dice and selects an outcome based on this probability, and we find out many turns later if that was a good move. 

Let the outcomes of a game state be represented by $y = 1 = \mathrm{UP}$ and $y = 0 = \mathrm{DOWN}$. We want to penalize bad moves and reward good moves. So how do we do this? Suppose, on a given game frame, the network chose to move the padel UP.  The probability of it making this decision is $P(y = 1 | X)$, "probability of UP given X". For future games, we want to either enhance this probability of UP, if it led to a good outcome, or discourage it if it led to a bad outcome. If we want to encourage it, we adjust our weights using the _gradient_ $\partial L / \partial W_{i}$, which tells us which direction to move the weights in to increase our probability of predicting UP. If we want instead to _discourage_ this move, we multiply the gradient by a negative factor, which moves the weights in the opposite direction. So, mathematically:

\begin{equation*}
\Delta W_{i} = R \left[\frac{\partial L}{\partial W_{i}} \right]
\end{equation*}

* R is the reward factor (+1 for a win, -1 for a loss). 


**Policy Gradients** is this very idea of choosing an action, computing its gradient, and modulating that with the rewards from a later state. 
As Andrej Karpathy puts it:
> *And that’s it: we have a stochastic policy that samples actions and then actions that happen to eventually lead to good outcomes get encouraged in the future, and actions taken that lead to bad outcomes get discouraged.* 


Where does the gradient come from? Our prediction problem can be represented as a case of binary classification, i.e. "should I predict UP or DOWN for this input?". Think of this part like a supervised learning task. We have X, and we want to predict the correct label y. We need a "loss function" that, when we minimize it, we are minimizing the error between the predicted and target labels. In this case we can use a variant of the [cross-entropy loss](http://neuralnetworksanddeeplearning.com/chap3.html) (see also [this](http://cs231n.github.io/neural-networks-2/#losses)):
\begin{equation*}
L = y\sigma(logit) + (1-y)(1 - \sigma(logit))
\end{equation*}

The gradients tell us how this loss changes as a function of the weights. We obtain these by taking partial derivates and using chain rule with good ole' Calculus:
\begin{align*}
\frac{\partial L}{\partial W_{2}} &= \left(\frac{\partial L}{\partial logit}\right) \left(\frac{\partial logit}{\partial W_{2}}  \right) \\
\frac{\partial L}{\partial W_{1}} &= \left(\frac{\partial L}{\partial logit}\right) \left(\frac{\partial logit}{\partial h}  \right) \left(\frac{\partial h}{\partial W_{1}}  \right) 
\end{align*}


For example, 
\begin{equation*}
\left(\frac{\partial L}{\partial logit}\right) = (y - \sigma(h)), \,\,\, \left(\frac{\partial logit}{\partial W_{2}}\right) = h, \,\,\, \mathrm{so} \frac{\partial L}{\partial W_{1}} =  (y - \sigma(h)) \cdot h
\end{equation*}

The notation below can be a bit confusing, so here is the shorthand:
* `dWi` = $\frac{\partial L}{\partial W_{i}}$ are the gradients
* `eph` are the hidden states `h` for this episode (ep)
* `epdlogp` is dlogp = $\frac{\partial L}{\partial logit}$ = $y - \sigma(h)$ for this episode (ep)

In [ ]:
def policy_backward(eph, epdlogp):
    """ backward pass. (eph is array of intermediate hidden states for the episode) """
    dW2 = np.dot(eph.T, epdlogp).ravel()
    dh = np.outer(epdlogp, model['W2'])
    dh[eph <= 0] = 0 # backprop relu
    dW1 = np.dot(dh.T, epx)
    return {'W1':dW1, 'W2':dW2}

### Rewards

If we win a game, should we encourage _every_ action that led to that win with a reward of +1? Not necessarily. The actions that happened in the distant past are probably not as important as the actions that immediately preceeded a win or a loss. 

So we typically use a 'discounted reward': The strength with which we encourage a sampled action is the weighted sum of all rewards afterwards, but later rewards are exponentially less important. 

\begin{equation*}
R_{t} = \sum^{\infty}_{k=0} \gamma^{k}r_{t+k}
\end{equation*}
* $\gamma$ is the discount factor, a number between 0 and 1
* $r_{t}$ is the reward for action $t$

Note: we'll typically calculate this at the end of an epoch. So it has the effect of "spreading" the sparse rewards over the actions of that epoch... e.g. 
\begin{equation*}
[0, 0, 0, ...,  0, 0, 0, 0, 0, 0, 1] \rightarrow [0, 0, 0, ... , 0.01, 0.06, 0.13, 0.38, 0.60, 0.83, 1.00]
\end{equation*}
...where each element is the reward for a particular action in chronological sequence. 

In [ ]:
def discount_rewards(r):
    """ take 1D float array of rewards and compute discounted reward """
    discounted_r = np.zeros_like(r)
    running_add = 0
    for t in reversed(range(0, r.size)):
        if r[t] != 0: running_add = 0 # reset the sum, since this was a game boundary (pong specific!)
        running_add = running_add * gamma + r[t]
        discounted_r[t] = running_add
    return discounted_r


### Training Loop
See comments for explanation. Note, we are interested in capturing the ball's motion, so we train on the difference between pixel frames. 
* For Pong, one episode is one complete set of games (an episode ends once one player has won 21 games). 
* For each episode, we have to track X, h, dlogp, rewards etc, for each game state (i.e. frame). 
* Note: for most frames, the reward will be zero, except for the last frame where it is +1 (win) or -1 (lose). 
  * Hence we must apply the discounted reward backwards through time.  
* We accumulate our gradients over a batch of `batch_size` episodes, before applying weight updates to our parameters (this helps smooth our updates). 
* We control our weight updates with a learning rate using the [rmsprop method](https://towardsdatascience.com/a-look-at-gradient-descent-and-rmsprop-optimizers-f77d483ef08b) (also [this](https://towardsdatascience.com/understanding-rmsprop-faster-neural-network-learning-62e116fcf29a) and [this](http://ruder.io/optimizing-gradient-descent/)). 

It can take MANY hours for the performance to begin to improve substantially. Keep in mind, we're training here on regular CPUs, this can be sped up considerably by switching to GPUs (see Tensorflow implementation listed in the next section). You should start to see small improvements to the 'running mean' as the model trains. This might only be a few decimal points after the first several undred episodes. 

In [ ]:
# *** INITIALIZE PONG ***
env = gym.make("Pong-v0")
observation = env.reset()

# *** INITIALIZE TRAINING VARIABLES ***
prev_x = None # used in computing the difference frame
xs,hs,dlogps,drs = [],[],[],[]
running_reward = None
reward_sum = 0
episode_number = 0

grad_buffer = { k : np.zeros_like(v) for k,v in model.items() } # update buffers that add up gradients over a batch
rmsprop_cache = { k : np.zeros_like(v) for k,v in model.items() } # rmsprop memory

history = {'episode_number': [], 'running_reward': [], 'reward_sum': []}

# *** RUN TRAINING LOOP ***
while True:
    if render: env.render() # shows the simulated pong games in a pop-up

    # --- preprocess the observation, set input to network to be difference image
    cur_x = prepro(observation)
    x = cur_x - prev_x if prev_x is not None else np.zeros(D)
    prev_x = cur_x

    # --- forward the policy network and sample an action from the returned probability
    aprob, h = policy_forward(x)
    action = 2 if np.random.uniform() < aprob else 3 # roll the dice!
    # note: action of 2 signals UP, action of 3 signals down, in env.step(action)...
    # ... these are like the specific game controller channels 

    # --- record various intermediates (needed later for backprop)
    xs.append(x) # observation
    hs.append(h) # hidden state
    y = 1 if action == 2 else 0 # a "fake label" that matches the action
    dlogps.append(y - aprob) # gradient that encourages the action that was taken to be taken 

    # -- step the environment and get new measurements
    observation, reward, done, info = env.step(action)
    reward_sum += reward
    drs.append(reward) # record reward (has to be done after we call step() to get reward for previous action)

    if done: # an episode finished
        episode_number += 1

        # --- stack together all inputs, hidden states, action gradients, and rewards for this episode
        epx = np.vstack(xs) # inputs...
        eph = np.vstack(hs) # hidden states...
        epdlogp = np.vstack(dlogps) # action gradients
        epr = np.vstack(drs) # rewards
        xs,hs,dlogps,drs = [],[],[],[] # reset array memory

        # --- compute the discounted reward backwards through time
        discounted_epr = discount_rewards(epr)
        # standardize the rewards to be unit normal (helps control the gradient estimator variance)
        discounted_epr -= np.mean(discounted_epr)
        discounted_epr /= np.std(discounted_epr)

        # --- apply the discounted reward to the gradient 
        # note: mathematically, it doesn't matter if we compute the gradient first and then
        # multiply it by R, or instead first multiply dL/dlogit by R and then compute gradient
        epdlogp *= discounted_epr 
        grad = policy_backward(eph, epdlogp)
        for k in model: grad_buffer[k] += grad[k] # accumulate gradient over batch

        # --- perform rmsprop parameter update every batch_size episodes
        if episode_number % batch_size == 0:
            for k,v in model.items():
                g = grad_buffer[k] # gradient
                rmsprop_cache[k] = decay_rate * rmsprop_cache[k] + (1 - decay_rate) * g**2
                model[k] += learning_rate * g / (np.sqrt(rmsprop_cache[k]) + 1e-5)
                grad_buffer[k] = np.zeros_like(v) # reset batch gradient buffer

        # --- book-keeping
        running_reward = reward_sum if running_reward is None else running_reward * 0.9 + reward_sum * 0.1
        print('resetting env. episode reward total was %f. running mean over past 10 episodes: %f' % (reward_sum, running_reward))
        if episode_number % save_rate == 0: pickle.dump(model, open('saved_model.p', 'wb'))
        history['episode_number'].append(episode_number)
        history['running_reward'].append(running_reward)
        history['reward_sum'].append(reward_sum)
        if episode_number % save_rate == 0: pickle.dump(history, open('history.p', 'wb'))   
        reward_sum = 0
        observation = env.reset() # reset env
        prev_x = None

    if reward != 0: # Pong has either +1 or -1 reward exactly when game ends.
        print ('ep %d: game finished, reward: %f' % (episode_number, reward) + ('' if reward == -1 else ' !!!!!!!!') )


***
### What to Expect
For this model
* Takes about 500 episodes to see if the mnodel is improving. 
* Takes about 8000 episodes to reach a point where the model is as good as computerized component (winning ~ half of the games). 

And here we've outlined a key challenge for reinforcement learning. Even though these networks can eventually surpass human performance, they're actually pretty poor learners compared to humans. We only need to play a couple games of pong to start to pick up on how to play it decently, whereas an AI needs to play thousands of games. 

Consider other learning tasks. When you first learn how to drive a car, you don't have to crash it a hundred times to know that you should stay on the road and in-between your lane lines. The reason is that you have other abstract models of your environment to guide you. Whereas an AI trained using vanilla Reinforcement Learning would fail many, many times before starting to succeed. 

RL still has a long way to go, but it shows promise when used in hybrid with other models, and may one day allow us to build more general-purpose AIs. 

***
### Ways to Improve
Building upon this basic implementation, we can achieve better performance, faster, with fewer training games:
* Tensorflow implementation: https://github.com/mrahtz/tensorflow-rl-pong
* Improve upon model using the Trust Region Policy Optimzation [(TRPO) method](https://arxiv.org/abs/1502.05477)
* Add [L2 Regularization](https://towardsdatascience.com/intuitions-on-l1-and-l2-regularisation-235f2db4c261) (also see [this])(https://openreview.net/forum?id=HkGmDsR9YQ).

* Another approach might be to create a simpler representation, i.e. give the network an inherent understanding of what a "paddle" or "ball" is. For example, we only need one vertical coordinate to track each paddle; for the ball we need its x-y coordinates as well as x-y velocities. Then instead of an 80x80 pixel input, we could only have 6 input features... 

***
## Plot a Trained Model's History

Note: an episode continues until one of the players reaches a score of 21. 

* A running reward of -21 means all games were lost. 
* Running reward of 0 means 50% of games were won. 
* Reward of 21 means all games were won. 

In [ ]:
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

history = pickle.load(open('history.p', 'rb'))

plt.plot(history['episode_number'], history['running_reward'])
plt.title('Training History')
plt.xlabel('Number of Training Episodes')
plt.ylabel('Average Reward Per Episode')
plt.show()


***
## Watching a Trained Model Play

In [ ]:
import gym
import pickle
import time
import numpy as np

model = pickle.load(open('trained_model.p', 'rb'))

FRAME_REFRESH_RATE = 0.025 # in seconds

D = 80 * 80 

def prepro(I):
    """ preproccess 210x160x3 uint8 game frame into 6400 (80x80) 1D float vector """
    I = I[35:195] # crop
    I = I[::2,::2,0] # downsample by factor of 2
    I[I == 144] = 0 # erase background (background type 1)
    I[I == 109] = 0 # erase background (background type 2)
    I[I != 0] = 1 # everything else (paddles, ball) just set to 1
    return I.astype(np.float).ravel()

def sigmoid(x): 
    return 1.0 / (1.0 + np.exp(-x)) # sigmoid "squashing" function to interval [0,1]

def policy_forward(x):
    h = np.dot(model['W1'], x)
    h[h<0] = 0 # ReLU nonlinearity
    logp = np.dot(model['W2'], h)
    p = sigmoid(logp)
    return p, h # return probability of taking action 2, and hidden state

env = gym.make("Pong-v0")
observation = env.reset()
prev_x = None # used in computing the difference frame

while True:
    env.render() 
    time.sleep(FRAME_REFRESH_RATE)

    # --- preprocess the observation, set input to network to be difference image
    cur_x = prepro(observation)
    x = cur_x - prev_x if prev_x is not None else np.zeros(D)
    prev_x = cur_x

    # --- forward the policy network and sample an action from the returned probability
    aprob, h = policy_forward(x)
    action = 2 if np.random.uniform() < aprob else 3 # roll the dice!
    y = 1 if action == 2 else 0 # a "fake label" that matches the action

    # -- step the environment 
    observation, reward, done, info = env.step(action)